In [40]:
import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import torch

In [85]:
pt = PyntCloud.from_file('../data/birmingham_block_0/segmented-cloud-subsampled.ply')

In [42]:
help(pt)

Help on PyntCloud in module pyntcloud.core_class object:

class PyntCloud(builtins.object)
 |  PyntCloud(points, mesh=None, structures=None, **kwargs)
 |  
 |  A Pythonic Point Cloud.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, points, mesh=None, structures=None, **kwargs)
 |      Create PyntCloud.
 |      
 |      Parameters
 |      ----------
 |      points: pd.DataFrame
 |          DataFrame of N rows by M columns.
 |          Each row represents one point of the point cloud.
 |          Each column represents one scalar field associated to its corresponding point.
 |      
 |      mesh: pd.DataFrame or None, optional
 |          Default: None
 |          Triangular mesh associated with points.
 |      
 |      structures: dict, optional
 |          Map key(base.Structure.id) to val(base.Structure)
 |      
 |      kwargs: custom attributes
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  add_scalar_field(self, name, **kwargs)
 |      Add one or multiple column

In [43]:
pt.points

,x,y,z,red,green,blue,scalar_R,scalar_G,scalar_B,scalar_Composite,scalar_Original_cloud_index,scalar_R_#1,scalar_G_#1,scalar_B_#1,scalar_Composite_#1,scalar_Classification
0,340.81250,365.43750,14.099998,103,109,112,103.0,109.0,112.0,108.000000,0.0,103.0,109.0,112.0,108.000000,0.0
1,340.37500,365.43750,14.160004,97,101,105,97.0,101.0,105.0,101.000000,0.0,97.0,101.0,105.0,101.000000,0.0
2,338.06250,365.21875,14.040001,105,112,121,105.0,112.0,121.0,112.666664,0.0,105.0,112.0,121.0,112.666664,0.0
3,341.28125,365.21875,14.020004,96,100,103,96.0,100.0,103.0,99.666664,0.0,96.0,100.0,103.0,99.666664,0.0
4,341.75000,364.96875,14.000000,84,89,91,84.0,89.0,91.0,88.000000,0.0,84.0,89.0,91.0,88.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107847,368.84375,228.31250,17.080002,51,52,51,51.0,52.0,51.0,51.333332,5.0,NaN,NaN,NaN,NaN,1.0
107848,369.18750,227.34375,17.980003,53,64,60,53.0,64.0,60.0,59.000000,5.0,NaN,NaN,NaN,NaN,1.0
107849,369.43750,226.21875,18.870003,91,120,52,91.0,120.0,52.0,87.666664,5.0,NaN,NaN,NaN,NaN,1.0
107850,370.46875,226.90625,14.470001,73,80,53,73.0,80.0,53.0,68.666664,5.0,NaN,NaN,NaN,NaN,1.0


In [44]:
pt.points['label'] = np.astype(pt.points['scalar_Classification'].values, int)

In [50]:
np.unique(pt.points['label'])

array([0, 1, 2, 3, 4, 5])

In [45]:
classes_count = pd.DataFrame(pt.points['label']).value_counts().reset_index().set_index('label')

In [46]:
classes_df = pd.DataFrame.from_dict({'label': [1, 2, 3, 4, 5, 0], 'name': ['building', 'green area', 'car', 'ground', 'tree', 'road']}).set_index('label')
classes_df

,name
label,
1,building
2,green area
3,car
4,ground
5,tree
0,road


In [47]:
counts = classes_df.join(classes_count)

In [48]:
counts

,name,count
label,,
1,building,48377
2,green area,9617
3,car,1883
4,ground,24442
5,tree,8420
0,road,15113


In [49]:
pt.points[['x', 'y', 'z']][:100]

,x,y,z
0,340.81250,365.43750,14.099998
1,340.37500,365.43750,14.160004
2,338.06250,365.21875,14.040001
3,341.28125,365.21875,14.020004
4,341.75000,364.96875,14.000000
...,...,...,...
95,344.06250,358.21875,13.950005
96,342.56250,355.53125,14.080002
97,399.03125,263.62500,15.040001
98,399.21875,263.68750,14.290001


In [52]:
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing
import torch
import torch.nn.functional as F
from torch_cluster import knn_graph
from torch_geometric.nn import global_max_pool


class PointNetLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')

        # Initialization of the MLP:
        # Here, the number of input features correspond to the hidden node
        # dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(Linear(in_channels + 3, out_channels),
                              ReLU(),
                              Linear(out_channels, out_channels))

    def forward(self, h, pos, edge_index):
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self, h_j, pos_j, pos_i):
        # h_j defines the features of neighboring nodes as shape [num_edges, in_channels]
        # pos_j defines the position of neighboring nodes as shape [num_edges, 3]
        # pos_i defines the position of central nodes as shape [num_edges, 3]

        input = pos_j - pos_i  # Compute spatial relation.

        if h_j is not None:
            # In the first layer, we may not have any hidden node features,
            # so we only combine them in case they are present.
            input = torch.cat([h_j, input], dim=-1)

        return self.mlp(input)  # Apply our final MLP.
    

class PointNet(torch.nn.Module):
    def __init__(self, n_classes):
        super().__init__()

        torch.manual_seed(12345)
        self.conv1 = PointNetLayer(3, 32)
        self.conv2 = PointNetLayer(32, 32)
        self.classifier = Linear(32, n_classes)

    def forward(self, pos, batch):
        # Compute the kNN graph:
        # Here, we need to pass the batch vector to the function call in order
        # to prevent creating edges between points of different examples.
        # We also add `loop=True` which will add self-loops to the graph in
        # order to preserve central point information.
        edge_index = knn_graph(pos, k=16, batch=batch, loop=True)

        # 3. Start bipartite message passing.
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        # 4. Global Pooling.
        h = global_max_pool(h, batch, size=h.shape[0])  # [num_examples, hidden_channels]

        # 5. Classifier.
        return self.classifier(h)


In [53]:
model = PointNet(n_classes=6)
print(model)

PointNet(
  (conv1): PointNetLayer()
  (conv2): PointNetLayer()
  (classifier): Linear(in_features=32, out_features=6, bias=True)
)


In [79]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset, Data

delta = 128

datalist = [
    Data(pos=torch.tensor(pt.points.loc[i: i+ delta, ['x', 'y', 'z']].values), y=torch.tensor(pt.points.loc[i: i + delta, ['label']].values, dtype=torch.long)) 
    for i in range(0, len(pt.points), delta)
]

train_loader = DataLoader(datalist, batch_size=10, shuffle=True)

In [80]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.

In [81]:
for data in train_loader:
    print(data.batch)
    break

tensor([0, 0, 0,  ..., 9, 9, 9])


In [82]:
def train(model, optimizer, loader):
    model.train()

    total_loss = 0
    for data in loader:
        optimizer.zero_grad()  # Clear gradients.
        logits = model(data.pos, data.batch)  # Forward pass.
        loss = criterion(logits, torch.flatten(data.y))  # Loss computation.
        loss.backward()  # Backward pass.
        optimizer.step()  # Update model parameters.
        total_loss += loss.item() * data.num_graphs

    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(model, loader):
    model.eval()

    total_correct = 0
    for data in loader:
        logits = model(data.pos, data.batch)
        pred = logits.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())

    return total_correct / len(loader.dataset)

In [83]:
torch.flatten(torch.tensor([[1, 2, 3]]))

tensor([1, 2, 3])

In [84]:
for epoch in range(1, 51):
    loss = train(model, optimizer, train_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([1290, 32])
torch.Size([1290, 1]) torch.Size([1290, 6])
torch.Size([1290, 3]) torch.Size([1290])
h shape: torch.Size([12

KeyboardInterrupt: 